# Разработка рекомендательной системы на Python

Можно выделить два основных типа рекомендательных систем.

**Content-based**

* Пользователю рекомендуются фильмы, похожие на те, которые он уже посмотрел.
* Похожести оцениваются по признакам содержимого объектов.
* Сильная зависимость от предметной области, полезность рекомендаций ограничена.

Коллаборативная фильтрация (**Collaborative Filtering**)

* Для рекомендации используется история оценок как самого пользователя, так и других пользователей.
* Более универсальный подход, часто дает лучший результат.
* Есть свои проблемы (например, холодный старт).

## Датасет

В данной работе используется MovieLens Dataset (Small). Посмотреть информацию или скачать датасет можно [отсюда](https://grouplens.org/datasets/movielens/).
Для дальнейшей работы применяется файл `ratings.csv`

In [1]:
import numpy as np
import pandas as pd

In [2]:
# загружаем датасет
df = pd.read_csv('ml-latest-small/ratings.csv')

In [3]:
# смотрим на структуру
df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [4]:
# выводим количество пользователей и фильмов
n_users = df['userId'].unique().shape[0]
n_items = df['movieId'].unique().shape[0]

In [5]:
print('Users: {}, Items: {}'.format(n_users, n_items))

Users: 610, Items: 9724


In [6]:
# чтобы можно было удобно работать дальше, необходимо отмасштабировать 
# значения в колонке movieId (новые значения будут в диапазоне от 1 до
# количества фильмов)
input_list = df['movieId'].unique()

def scale_movie_id(input_id):
    return np.where(input_list == input_id)[0][0] + 1

df['movieId'] = df['movieId'].apply(scale_movie_id)

In [7]:
from sklearn import model_selection as cv

# делим данные на тренировочный и тестовый наборы
train_data, test_data = cv.train_test_split(df, test_size=0.20)

## Memory-Based Collaborative Filtering

Memory-Based Collaborative Filtering подходы можно разделить на две части: **user-item filtering** and **item-item filtering**.

В user-item filtering мы:

1. Берём исходного пользователя
2. Находим группу пользователей, которая максимально похожа на него (основываясь, например, оценках) и узнаём, какие фильмы понравились этой группе.
3. Нашему исходному пользователю рекомендуем фильмы, которые нравятся найденной группе пользователей.

На входе пользователь, на выходе – рекомендация фильмов для данного пользователя.

В item-item filtering мы:
    
1. Берём какой-либо фильм
2. Находим пользователей, которым нравится этот фильм
3. Смотрим на фильмы, которые нравятся найденным пользователям и выводим их в качестве рекомендации к исходному товару

На входе фильм, на выходе – рекомендация в виде похожих фильмов.

* Item-Item Collaborative Filtering: "Пользователям, которым нравится данный фильм, может так же понравиться это ..."
* User-Item Collaborative Filtering: "Похожим на вас пользователям нравится это ..."

В обоих случаях нам необходимо создать user-item матрицу, которая будет выглядеть следующим образом:

|       | Item1 | Item2 | Item3 |
|-------|-------|-------|-------|
| User1 |   5   |   3   |   4   |
| User2 |   4   |   0   |   0   |
| User3 |   0   |   0   |   0   |

В ячейках матрицы будет записана информация об оценке фильма $m$ пользователя $n$.

In [8]:
# создаём две user-item матрицы – для обучения и для теста
train_data_matrix = np.zeros((n_users, n_items))
for line in train_data.itertuples():
    train_data_matrix[line[1] - 1, line[2] - 1] = line[3]  

test_data_matrix = np.zeros((n_users, n_items))
for line in test_data.itertuples():
    test_data_matrix[line[1] - 1, line[2] - 1] = line[3]

После того, как мы построим данную матрицу, на её основе необходимо будет рассчитать две новые матрицы с коэффициентами сходства (похожести, близости) для пользователей и для фильмов.

В качестве метрики близости в данной работе используется косинусное расстояние между векторами пользователей (фильмов).

Формула для пользователей:

$$ s_{u}^{cos}(u_k, u_a) = \frac{u_k \cdot  u_a}{\left \| u_k \right \| \left \| u_a \right \|} = \frac{\sum x_{k,m} x_{a,m}}{\sqrt{\sum x_{k,m}^2 \sum x_{a,m}^2}} $$

Формула для фильмов:

$$ s_{u}^{cos}(i_m, i_b) = \frac{i_m \cdot  i_b}{\left \| i_m \right \| \left \| i_b \right \|} = \frac{\sum x_{a,m} x_{a,b}}{\sqrt{\sum x_{a,m}^2 \sum x_{a,b}^2}} $$

In [9]:
from sklearn.metrics.pairwise import pairwise_distances

# считаем косинусное расстояние для пользователей и фильмов
user_similarity = pairwise_distances(train_data_matrix, metric='cosine')
item_similarity = pairwise_distances(train_data_matrix.T, metric='cosine') # обратите внимание на транспонирование при работе с item-based

Матрица "похожести" для пользователей имеет следующий вид (аналогичную структуру имеет и матрицы для фильмов):

|       | User1 | User1 | User3 |
|-------|-------|-------|-------|
| User1 |   0   | 0.87  | 0.99  |
| User2 |   123   |   0   |   123123   |
| User3 |   123   |   123   |   0  |

Для предсказания в user-based CF необходимо применить следующую формулу:

$$ \hat{x}_{k,m} = \overline{x}_k + \frac{\sum_{u_a} sim_u(u_k, u_a)(x_{a,m} - \overline{x}_{u_a})}{\sum_{u_a} \left | sim_u(u_k, u_a) \right |} $$

Для item-based CF:

$$ \hat{x}_{k,m} = \frac{\sum_{i_b} sim_i(i_m, i_b)(x_{k,b})}{\sum_{i_b} \left | sim_i(i_m, i_b) \right |} $$

In [18]:
def predict(train_data_matrix, similarity, top):
    # Матрица для хранения индексов top
    top_similar = np.zeros((n_users, top))
    
    for i in range(n_users):
        # Похожесть пользователя i 
        user_sim = user_similarity[i]
        # Сортировка индексов наиболее похожих пользователей
        top_sim_users = user_sim.argsort()[1:top + 1]
        # По каждому из наиболее похожих пользователей
        for j in range(top):
            # Индекс j-пользователя (u`) для пользователя i (u)
            top_similar[i, j] = top_sim_users[j]
    # Модуль каждого элемента в матрице похожести
    abs_sim = np.abs(user_similarity)
    # Хранение прогнозируемых рейтингов
    pred = np.zeros((n_users, n_items))
    
    for i in range(n_users):
        # Индексы наиболее похожих пользователей
        indexes = top_similar[i].astype(int)
        # Сходноства между i и top похожими пользователями
        numerator = user_similarity[i][indexes]
        # Числитель формулы
        product = numerator.dot(train_data_matrix[indexes])
        # Знаменатель формулы
        denominator = abs_sim[i][top_similar[i].astype(int)].sum()
        
        pred[i] = product / denominator
    return pred

In [20]:
similarity = user_similarity
prediction = predict(train_data_matrix, similarity, 7)

Для оценки качества предсказания используем метрику RMSE (Root Mean Square Error, cреднеквадратичная ошибка):

$$ RMSE = \sqrt{\frac{1}{N}\sum (x_i - \hat{x}_i)^2} $$

In [23]:
from sklearn.metrics import mean_squared_error
from math import sqrt

def rmse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten() 
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, ground_truth))

In [26]:
print('RMSE: ' + str(rmse(prediction, test_data_matrix)))

RMSE: 2.765044389166193


## Формулы для построения рекомендательных систем

Примечание: формулы ниже описывают оценку в коллаборативной фильтрации (user-based). Content-based работает по аналогичному принипу, только вместо $u$ $u'$ подставляются $i$ $i'$.

### Наивные рекомендации

Наивная рекомендательная система считает предсказанную оценку пользователя `u` фильму `i` по формуле:

$$
r_{u, i}=\frac{\sum_{u^{\prime} \in U} r_{u^{\prime}, i}}{N}
$$

где

$N$ — количество пользователей, похожих на пользователя $u$,

$U$ — множество из $N$ похожих пользователей,

$u'$ — пользователь, похожий на пользователя $u$ (из множества $U$),

$r_{u', i}$ — оценка пользователя $u'$ фильму $i$,

$r_{u,i}$ — предсказанная оценка фильма $i$.

По этой формуле предсказываемая оценка фильму $i$ пользователя u равняется средней оценке фильма $i$ от $N$ пользователей, наиболее похожих на пользователя $u$.

### Рекомендации с учётом средних оценок похожих пользователей

Менее наивная реализация требует участия матрицы “похожести” и оценок “похожих” пользователей. Формула вычисления предсказанной оценки:

$$
r_{u, i}=\frac{\sum_{u^{\prime} \in U} \operatorname{simil}\left(u, u^{\prime}\right) r_{u^{\prime}, i}}{\sum_{u^{\prime} \in U}\left|\operatorname{simil}\left(u, u^{\prime}\right)\right|}
$$

где

$simil(u, u')$ — “похожесть” пользователя $u$ и пользователя $u'$,

$r_{u', i}$ — оценка пользователя $u’$ из $U$ фильму $i$,

$U, u'$ — аналогичны предыдущей формуле.

То есть предсказываемая оценка фильму будет равна сумме произведений “похожести” пользователя на его оценку по всем наиболее похожим пользователям.

### Рекомендации на основе средних оценок пользователей и матрицы “похожести”

Третья реализация зависит от оценок, которые пользователь ставил ранее (точнее, от средней оценки по всем оценённым пользователем фильмам), средних оценок “похожих” пользователей и коэффициентов “похожести”:

$$
r_{u, i}=\bar{r}_u+\frac{\sum_{u^{\prime} \in U} \operatorname{simil}\left(u, u^{\prime}\right)\left(r_{u^{\prime}, i}-\bar{r}_{u^{\prime}}\right)}{\sum_{u^{\prime} \in U}\left|\operatorname{simil}\left(u, u^{\prime}\right)\right|}
$$

где $\bar{r}$ — средняя оценка соответствующего пользователя, а остальные переменные формулы уже обсуждались выше.